<h3 align="center">
实验五: 非平衡电桥的应用
</h3>

请将第一个 python 单元格中的内容复制，**和你的实验报告的完整照片**提交给一个大语言模型（我们建议使用 Google Gemini 2.5 Pro 或者等价逻辑能力的LLM）。将其生成的 Python 代码复制并替换第一个单元格，此时文件应当没有报错。运行所有单元格以生成数据分析报告。

In [ ]:
"""
You are a lab agent, and your task is to extract data faithfully from the images provided. If no image is provided, please tell the user to upload photos of his or her lab report.

Read the following instructions carefully before you answer.
- The images should contain clear, handwritten text. If any part is unclear, ask the user to supply a clearer image or transcribe the unclear part.
- Pay special attention to the unit used by the user. Compare measure results with standard ones, and convert them if necessary. Automatically adjust magnitudes, and inform the user of the adjustments in text before the yaml snippet.
- When writing code you are expected to follow the given format strictly. If no warnings or failures occur, you should output a single yaml snippet, enclosed within triple backticks. You need not add extra comments to your answer. Do not output any text outside the code block except warnings or failure messages defined above.

Data related to NTC (aka temperature sensor):
- Resistance at 0°C, a value from 2000-10000 ohm. Extract to `ntc_r0: float`. Unit: ohm.
- Current for calibration, a value around 0.1mA. Extract to `ntc_ci: float`. Unit: mA.
- A table of temperature-voltage pairs. Extract to `ntc_temp: list[float]` and `ntc_volt: list[float]`. Unit: °C and mV.

Data related to Pt-100 (aka platinum resistance thermometer):
- Resistance at 0°C, a value around 100 ohm. Extract to `pt100_r0: float`. Unit: ohm.
- Current for calibration, a value around 1-10mA. Extract to `pt100_ci: float`. Unit: mA.
- A table of temperature-voltage pairs. Extract to `pt100_temp: list[float]` and `pt100_volt: list[float]`. Unit: °C and mV.

Sample output format:
```
ntc-data:
  # Rp at 0°C to balance the bridge (Ω)
  rp: null
  
  # Current through the bridge at 100°C calibration (mA)
  ci: null
  
  # Temperature readings (°C)
  temperature: []
  
  # Voltage readings (mV)
  voltage: []

pt100-data:
  # Rp at 0°C to balance the bridge (Ω)
  rp: null
  
  # Current through the bridge at 100°C calibration (mA)
  ci: null
  
  # Temperature readings (°C)
  temperature: []
  
  # Voltage readings (mV)
  voltage: []
```
"""

In [ ]:
import yaml

try:
    with open("data.yaml", "r") as f:
        data = yaml.safe_load(f)
except FileNotFoundError:
    raise FileNotFoundError("data.yaml not found in current directory. Please make sure the file exists.")
except yaml.YAMLError as e:
    raise ValueError(f"Error parsing YAML file: {e}. Please ensure that the agent returned a valid YAML snippet.")

try:
    ntc_data = data["ntc-data"]
    ntc_rp = ntc_data["rp"]
    ntc_ci = ntc_data["ci"]
    ntc_temp = ntc_data["temperature"]
    ntc_volt = ntc_data["voltage"]

    pt100_data = data["pt100-data"]
    pt100_rp = pt100_data["rp"]
    pt100_ci = pt100_data["ci"]
    pt100_temp = pt100_data["temperature"]
    pt100_volt = pt100_data["voltage"]
except KeyError as e:
    raise KeyError(f"Missing expected key in YAML data: {e}. Please ensure all required fields are present.")

In [4]:
from rich.table import Table
from rich.console import Console
from math import log

def get_resistance_from_bridge(ci_ma: float, volt_mv: float, r_std: float, r_p: float) -> float:
    return (
        ci_ma * 1e-3 * r_std * r_p + (2 * r_std + r_p) * volt_mv * 1e-3
    ) / (ci_ma * 1e-3 * r_std - volt_mv * 1e-3)

ntc_r12 = 1000

ntc_kelvin = [t + 273.15 for t in ntc_temp]
ntc_resistance_a = [
    get_resistance_from_bridge(ntc_ci, v, ntc_r12, ntc_rp) for v in ntc_volt
]
ntc_resistance_b = [
    get_resistance_from_bridge(ntc_ci, -v, ntc_r12, ntc_rp) for v in ntc_volt
]
ntc_resistance = ntc_resistance_a if (
    ntc_resistance_a[0] > ntc_resistance_a[-1]
) else ntc_resistance_b

ntc_t_inv = [1 / t for t in ntc_kelvin]
ntc_ln_r = [log(r) for r in ntc_resistance]

table = Table(title="NTC Data")
table.add_column("Temperature (°C)", justify="right", style="cyan", no_wrap=True)
table.add_column("Voltage (mV)", justify="right", style="yellow")
table.add_column("Resistance (Ohm)", justify="right", style="white")
table.add_column("1/T (1e-3/K)", justify="right", style="white")
table.add_column("ln(R)", justify="right", style="white")
for t, v, r, t_inv, ln_r in zip(ntc_temp, ntc_volt, ntc_resistance, ntc_t_inv, ntc_ln_r):
    table.add_row(f"{t:.2f}", f"{v:.2f}", f"{r:.1f}", f"{1000 * t_inv:.3f}", f"{ln_r:.3f}")
console = Console()
console.print(table)

                                  NTC Data                                   
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Temperature (°C) ┃ Voltage (mV) ┃ Resistance (Ohm) ┃ 1/T (1e-3/K) ┃ ln(R) ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━┩
│            31.00 │        55.43 │           2443.2 │        3.288 │ 7.801 │
│            40.00 │        64.04 │           1921.8 │        3.193 │ 7.561 │
│            50.00 │        74.42 │           1351.0 │        3.095 │ 7.209 │
│            60.00 │        83.80 │            882.5 │        3.002 │ 6.783 │
│            70.00 │        90.05 │            592.3 │        2.914 │ 6.384 │
│            80.00 │        95.05 │            371.6 │        2.832 │ 5.918 │
│            90.00 │        98.40 │            229.1 │        2.754 │ 5.434 │
│           100.00 │       100.63 │            136.6 │        2.680 │ 4.917 │
└──────────────────┴──────────────┴──────────────────┴──────────────┴───────┘